## OPTECH. Use Case 1
### Phase 1
#### Based on paper: X. Liu, B. Wang  and Z. Yang, "Virtual Network Embedding Based on Topology Potential, " Entropy 2018, 20(12), 941.

In [ ]:
# Libraries needed
import networkx as nx
import math
import random
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd


import pickle
import sqlite3

In [ ]:
!pip install Pyomo
!apt-get install -y -qq glpk-utils

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.core import Var

## SCENARIO


## MODEL

In [ ]:
# Create a mathematical model (MILP)
model = ConcreteModel()

# S: set of switches
model.S = Set(initialize=['s{}'.format(i) for i in range(1,11)])
# C: set of controller types
model.C = Set(initialize=['c1','c2','c3','c4'])
# L: set of link types
model.L = Set(initialize=['l1','l2','l3'])
# P: set of possible locations to install the controllers
model.P = Set(initialize=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10'])

#################
## Parameters  ##
#################
alpha_values = {'c1':8, 'c2':32, 'c3':64, 'c4':128}

mu_values    = {'c1':2500, 'c2':4000, 'c3':8000, 'c4':15000}

k_values = {'c1':1200, 'c2':2500, 'c3':6500, 'c4':12000}

phi_c_values = {'c1':20, 'c2':15, 'c3':10, 'c4':6}

sigma_values = {}
for s in model.S:
    sigma_values[s] = random.randint(100, 999)

# σ^s: number of packets sent to the controller by switch s
model.sigma = Param(model.S, initialize=sigma_values)

# α^c: number of ports available for each controller type c ∈ C:
model.alpha = Param(model.C, initialize=alpha_values)

# μ^c: number of packets/second that a controller of type c can process
model.mu = Param(model.C, initialize=mu_values)

# κ^c: price ($) of a controller type c
model.k = Param(model.C, initialize=k_values)

# φ^c: number of available controllers of type c
model.phi_c = Param(model.C, initialize=phi_c_values)

#Link Parameters
omega_values = {'l1':100, 'l2':1000, 'l3':10000}
phi_l_values = {'l1':0.25, 'l2':0.63, 'l3':29}
model.omega = Param(model.L, initialize=omega_values)
model.phi_l = Param(model.L, initialize=phi_l_values)

#################
## Constants  ##
#################

# β: packet size (in bytes) for each packet sent to the controller
model.beta = Param(initialize= 150)

# γ: maximum delay (in ms) allowed in the network for flow-setup
model.gamma = Param(initialize= 250)

# δ: average time (in ms) taken to process a packet by any controller
model.delta = Param(initialize= 0.001)
################
## Variables  ##
################

# x_cp: binary variable = 1 if a controller of type c is installed at location p
model.x = Var(model.C, model.P, within=Binary, initialize=0)

# v_sp^l: binary variable = 1 if a link of type l is installed between switch s and controller at location p
model.v = Var(model.S, model.P, model.L, within=Binary, initialize=0)

# z_pq^l: binary variable = 1 if location p is connected to location q with a link type l
model.z = Var(model.P, model.P, model.L, within=Binary, initialize=0)

###############
## DISTANCIA ##
###############
switch_coords = {
    's1': (0,0), 's2': (1,0), 's3': (2,0), 's4': (3,0), 's5': (4,0),
    's6': (0,1), 's7': (1,1), 's8': (2,1), 's9': (3,1), 's10':(4,1)
}
location_coords = {
    'p1':(0,2), 'p2':(1,2), 'p3':(2,2), 'p4':(3,2), 'p5':(3,2), 'p6':(2,1), 'p7':(1,4), 'p8':(4,3), 'p9':(3,3), 'p10':(1,1)
}


def euclidean_dist(a, b):
    return math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)

# Initialize dist(s,p)???????????????????
dist_sp_init = {}
for s in model.S:
    for p in model.P:
        dist_sp_init[(s,p)] = euclidean_dist(switch_coords[s], location_coords[p])

# Initialize dist(p,q)?????????????
dist_pq_init = {}
for p in model.P:
    for q in model.P:
        if p != q:
            dist_pq_init[(p,q)] = euclidean_dist(location_coords[p], location_coords[q])

model.dist_sp = Param(model.S, model.P, initialize=dist_sp_init)
model.dist_pq = Param(model.P, model.P, within=Any, default=0, initialize=dist_pq_init)

############
## MODEL  ##
############

# COST FUNCTIONS

# OBJECTIVE FUNCION
def Objective_rule(model):

    Cc = sum(model.k[c]*model.x[c,p] for c in model.C for p in model.P)
    Cl = sum(model.phi_l[l]*model.dist_sp[s,p]*model.v[s,p,l] for l in model.L for s in model.S for p in model.P)
    Ct = sum(model.phi_l[l]*model.dist_pq[p,q]*model.z[p,q,l] for l in model.L for p in model.P for q in model.P if p<q)

    return Cc + Cl + Ct

model.Objective = Objective(rule=Objective_rule, sense=minimize)


# CONSTRAINTS
'''CONSTRAINT 1 '''
def C1_rule(model, p):
    return sum(model.x[c,p] for c in model.C) <= 1
model.C1 = Constraint(model.P, rule=C1_rule)

'''CONSTRAINT 2 '''
def C2_rule(model, p): #linealizar model.alpha*model.x
    return sum(model.z[p,q,l] + model.z[q,p,l] for q in model.P for l in model.L if q != p) + sum(model.v[s,p,l] for s in model.S for l in model.L) <= sum(model.alpha[c]* model.x[c,p] for c in model.C)
model.C2 = Constraint(model.P, rule=C2_rule)
'''CONSTRAINT 3 '''
def C3_rule(model, s):
    return sum(model.v[s,p,l] for l in model.L for p in model.P) == 1
model.C3 = Constraint(model.S, rule=C3_rule)
'''CONSTRAINT 4 '''
def C4_rule(model, p):
        return sum(model.sigma[s]*model.v[s,p,l] for s in model.S for l in model.L) <= sum(model.mu[c]*model.x[c,p] for c in model.C)
model.C4 = Constraint(model.P, rule=C4_rule)
'''CONSTRAINT 5 '''
def C5_rule(model, c):
    return sum(model.x[c,p] for p in model.P) <= model.phi_c[c]
model.C5 = Constraint(model.C, rule=C5_rule)
'''CONSTRAINT 6 '''
def C6_rule(model, s): # asumo 1Mbps
    return sum(model.omega[l]*125000*model.v[s,p,l] for p in model.P for l in model.L) >= model.sigma[s]*model.beta
model.C6 = Constraint(model.S, rule=C6_rule)
'''CONSTRAINT 7 '''
def C7_rule(model, s):
    return sum((2*((model.beta.value*8*1000)/(model.omega[l]*1e6)) + 2*(model.dist_sp[s,p]/2e5) + model.delta)*model.v[s,p,l]for p in model.P for l in model.L) <= model.gamma
model.C7 = Constraint(model.S, rule=C7_rule)
'''CONSTRAINT 8 '''
def C8_rule(model, p, q):
    if p < q:
        return sum(model.x[c,p] for c in model.C)+ sum(model.x[c,q] for c in model.C) <= sum(model.z[p,q,l] for l in model.L) + sum(model.z[q,p,l] for l in model.L) + 1
    else:
        return Constraint.Skip
model.C8 = Constraint(model.P, model.P, rule=C8_rule)


## SOLVE THE MODEL

In [ ]:
solver = SolverFactory('glpk', executable = '/usr/bin/glpsol')
results=solver.solve(model)
model.solutions.load_from(results)

## PRINT RESULTS

In [ ]:
installed_controllers = [(c, p) for c in model.C for p in model.P if model.x[c,p].value == 1]
num_controllers = len(installed_controllers)

inter_links = [(p,q,l) for p in model.P for q in model.P for l in model.L if p<q and model.z[p,q,l].value == 1]
switch_links = [(s,p,l) for s in model.S for p in model.P for l in model.L if model.v[s,p,l].value == 1]

total_cost = model.Objective()
total_packets = sum(model.sigma[s] for s in model.S)
cpu_time = results.solver.time
problem_number = 1

data = {
    'Problem': [problem_number],
    '|S|': [len(model.S)],
    '|P|': [len(model.P)],
    'Installed_Controllers': [num_controllers],
    'Inter-Controller Links': [len(inter_links)],
    'Switch-Controller Links': [len(switch_links)],
    'Total_Links': [len(inter_links) + len(switch_links)],
    'Packets': [total_packets],
    'Cost($)': [total_cost],
    'CPU_Time(s)': [cpu_time]
}

df_results = pd.DataFrame(data)

print("=== RESULTADOS DEL PROBLEMA ===")
print(f"- Número de switches (|S|): {len(model.S)}")
print(f"- Número de posibles ubicaciones (|P|): {len(model.P)}")
print(f"- Controladores instalados: {num_controllers}")
print(f"- Enlaces entre controladores: {len(inter_links)}")
print(f"- Enlaces switch-controlador: {len(switch_links)}")
print(f"- Paquetes totales: {total_packets}")
print(f"- Costo total: {total_cost}")
print(f"- Tiempo de CPU (s): {cpu_time}")
print()



=== RESULTADOS DEL PROBLEMA ===
- Número de switches (|S|): 10
- Número de posibles ubicaciones (|P|): 10
- Controladores instalados: 2
- Enlaces entre controladores: 0
- Enlaces switch-controlador: 10
- Paquetes totales: 3673
- Costo total: 2402.971587379343
- Tiempo de CPU (s): 3.907163619995117

